In [1]:
# Simulador Interactivo de Titulizaciones – SEC-IRBA y SEC-SA

Este notebook permite simular el comportamiento de los Activos Ponderados por Riesgo (RWA) bajo los enfoques SEC-IRBA y SEC-SA del marco CRR/Basilea III. Los parámetros pueden modificarse de forma interactiva para evaluar sensibilidad.

## Variables

### Riesgo de Crédito
- **PD**: Probabilidad de incumplimiento.
- **LGD**: Pérdida en caso de incumplimiento.
- **R**: Correlación regulatoria según exposición.

### Titulización
- **A**: Punto de attachment del tramo.
- **D**: Punto de detachment del tramo.
- **M**: Madurez residual (años).
- **STS**: Si la titulización cumple con criterios de simplicidad y transparencia.

### Métodos
- **SEC-IRBA**: Usa modelos internos (IRB).
- **SEC-SA**: Usa ponderaciones estándar.


SyntaxError: invalid syntax (3368739136.py, line 3)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import ipywidgets as widgets
from IPython.display import display

# Función K_IRB
def calculate_K_irb(PD, LGD, exposure_type='corporate'):
    if exposure_type == 'corporate':
        R = 0.12 * (1 - np.exp(-50 * PD)) / (1 - np.exp(-50)) + 0.24 * (1 - ((1 - np.exp(-50 * PD)) / (1 - np.exp(-50))))
    elif exposure_type == 'retail':
        R = 0.03
    else:
        raise ValueError("Tipo de exposición no soportado.")
    numerator = norm.ppf(PD) + np.sqrt(R) * norm.ppf(0.999)
    denominator = np.sqrt(1 - R)
    capital_requirement = LGD * norm.cdf(numerator / denominator) - PD * LGD
    return capital_requirement

# SEC-IRBA
def compute_sec_irba(K_irb, A, D, M):
    W = D - A
    capital_req = K_irb * W * M
    return 12.5 * capital_req

# SEC-SA simplificado
def calculate_K_sa(weights, risk_weights):
    return sum(w * rw for w, rw in zip(weights, risk_weights)) / 100

def compute_sec_sa(K_sa, A, D, M, sts=False):
    p = 0.5 if sts else 1.0
    W = D - A
    capital_req = p * K_sa * W * M
    return 12.5 * capital_req


In [ ]:
pd_slider = widgets.FloatSlider(value=0.05, min=0.001, max=0.2, step=0.001, description='PD:')
lgd_slider = widgets.FloatSlider(value=0.45, min=0.0, max=1.0, step=0.01, description='LGD:')
method_selector = widgets.Dropdown(options=['SEC-IRBA', 'SEC-SA'], value='SEC-IRBA', description='Método:')
sts_checkbox = widgets.Checkbox(value=False, description='STS')
tranche_A = widgets.FloatSlider(value=0.00, min=0.00, max=0.99, step=0.01, description='A:')
tranche_B = widgets.FloatSlider(value=0.05, min=0.01, max=1.00, step=0.01, description='B:')
tranche_C = widgets.FloatSlider(value=0.20, min=0.01, max=1.00, step=0.01, description='C:')

def update_plot(PD, LGD, method, sts, A, B, C):
    pd_values = np.linspace(0.001, 0.20, 100)
    M_values = [1, 3, 5]
    rwa_results = {M: [] for M in M_values}
    tranche_bounds = {'A': A, 'B': B, 'C': C}

    for pd in pd_values:
        if method == 'SEC-IRBA':
            K_ir = calculate_K_irb(pd, LGD)
        else:
            K_ir = calculate_K_sa([0.33, 0.33, 0.34], [75, 100, 150])

        for M in M_values:
            rwa_A = compute_sec_irba(K_ir, 0, tranche_bounds['A'], M) if method == 'SEC-IRBA' else compute_sec_sa(K_ir, 0, tranche_bounds['A'], M, sts)
            rwa_B = compute_sec_irba(K_ir, tranche_bounds['A'], tranche_bounds['B'], M) if method == 'SEC-IRBA' else compute_sec_sa(K_ir, tranche_bounds['A'], tranche_bounds['B'], M, sts)
            rwa_C = compute_sec_irba(K_ir, tranche_bounds['B'], tranche_bounds['C'], M) if method == 'SEC-IRBA' else compute_sec_sa(K_ir, tranche_bounds['B'], tranche_bounds['C'], M, sts)
            total_rwa = rwa_A + rwa_B + rwa_C
            rwa_results[M].append(total_rwa)

    plt.figure(figsize=(10, 6))
    for M in M_values:
        plt.plot(pd_values, rwa_results[M], label=f'M = {M} años')
    plt.xlabel('PD')
    plt.ylabel('RWA total (%)')
    plt.title(f'Titulización – Método {method}')
    plt.grid(True)
    plt.legend()
    plt.show()

ui = widgets.VBox([
    method_selector, sts_checkbox,
    widgets.HBox([pd_slider, lgd_slider]),
    widgets.HBox([tranche_A, tranche_B, tranche_C])
])
out = widgets.interactive_output(update_plot, {
    'PD': pd_slider,
    'LGD': lgd_slider,
    'method': method_selector,
    'sts': sts_checkbox,
    'A': tranche_A,
    'B': tranche_B,
    'C': tranche_C
})
display(ui, out)


Output()